In [12]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain-community langchainhub langgraph langchain_groq groq


In [13]:
pip install langchain langchain_groq groq

In [7]:
pip install pyprojroot gradio

In [5]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

In [180]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM users LIMIT 10;")

sqlite
['users']


"[(111, 'Farah Said', 'Farah@gmail.com', 'Product Management', 'Product Management Consultant with extensive experience in developing product strategies, analyzing market needs, and designing innovative solutions that meet customer expectations. Focused on achieving added value by guiding teams to deliver high-quality products, while ensuring they align with...', 5), (112, 'Moayad Khalid', 'MK@gmail.com', 'E-commerce Strategist', 'An e-commerce strategy consultant with extensive experience in building and developing successful online stores. Focuses on designing integrated strategies to boost sales, improve customer experience, and increase brand awareness online.', 4), (113, 'Anas Malik', 'AM@gmail.com', 'Cloud Computing', 'A dedicated cloud computing consultant with experience in designing and implementing innovative solutions that meet the needs of businesses. Focused on helping clients transition to cloud infrastructure, ensuring improved efficiency, reduced costs and enhanced secu

In [ ]:
db_path = "/content/data/sqldb1.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
chain = create_sql_query_chain(llm, db)


In [22]:
response1 = chain.invoke({"question": "Recommend me a Data Scientist consultant and a UX/UI Designer and tell me about them"})
sql_query = response1.split("SQLQuery:")[-1].strip()
print(sql_query)
query = db.run(sql_query)

SELECT "username", "about" 
FROM users 
WHERE "role" IN ('Data Scientist', 'UX/UI Designer') 
ORDER BY "rate" DESC 
LIMIT 2;


In [165]:
query

"[('Omar Al-Farsi', 'Data scientist with a deep understanding of statistical analysis, machine learning, and big data technologies. Experienced in leveraging data to build predictive models and data-driven solutions that help businesses make informed decisions and optimize operations.'), ('Noura Al-Sabah', 'UX/UI designer passionate about creating intuitive, user-centered designs. Expertise in wireframing, prototyping, and user research to enhance the user experience. Adept at translating complex requirements into engaging, accessible digital interfaces that meet business and user needs.')]"

In [17]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, create_sql_query_chain
from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase

warnings.filterwarnings("ignore")

db_path = "/content/data/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

llm = ChatGroq(
    model='llama3-70b-8192',
    temperature=0,
    groq_api_key=""
)

chain = create_sql_query_chain(llm, db)


FirstResponse = chain.invoke({"question": "recommend me a consultant, i want to test the Security of my app"})
sql_query = FirstResponse.split("SQLQuery:")[-1].strip()
print(sql_query)
query = db.run(sql_query)
PROMPT_TEMPLATE =    """
Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result:
Answer: """
language = "Arabic"
prompt = PromptTemplate(
    input_types={'question': 'string', query: 'string'},
    template=PROMPT_TEMPLATE
)

chain1 = LLMChain(llm=llm, prompt=prompt)

FinalResponse = chain1.invoke({"question": FirstResponse, "query": query})
FinalResponse['text']

SELECT "username", "about" FROM users WHERE "role" LIKE '%Security%' OR "about" LIKE '%Security%' LIMIT 5;


'Based on your query, I recommend the following two consultants to test the security of your app:\n\n1. Anas Malik - Although Anas is a cloud computing consultant, his profile mentions that he ensures "enhanced security" in his cloud infrastructure solutions, which could be relevant to your app\'s security testing needs.\n2. Maya Hassan - Maya is a dedicated cybersecurity expert with experience in designing and implementing security solutions, risk assessment, vulnerability management, and compliance with industry regulations. She seems like a great fit to test the security of your app.\n\nPlease reach out to either of them to discuss your app\'s security testing requirements.'

In [21]:
import gradio as gr
from langchain_community.utilities import SQLDatabase
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, create_sql_query_chain
from langchain_groq import ChatGroq

db_path = "/content/data/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

llm = ChatGroq(
    model='llama3-70b-8192',
    temperature=0,
    groq_api_key=""
)

sql_chain = create_sql_query_chain(llm, db)

PROMPT_TEMPLATE = """
Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:
"""

prompt = PromptTemplate(
    input_variables=['question', 'query', 'result'],
    template=PROMPT_TEMPLATE
)
answer_chain = LLMChain(llm=llm, prompt=prompt)

def process_query(user_question):
    first_response = sql_chain.invoke({"question": user_question})
    sql_query = first_response.split("SQLQuery:")[-1].strip()


    query_result = db.run(sql_query)


    final_response = answer_chain.invoke({
        "question": user_question,
        "query": sql_query,
        "result": query_result
    })

    return final_response['text']

iface = gr.Interface(
    fn=process_query,
    inputs="text",
    outputs="text",
    title="Khabeer AI Recommender",
    description="Ask questions related to any Consultant, you will get answers powered by Khabeer AI."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fef8737c50b3a548f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
